## Загрузим нужные библиотеки

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np 

%matplotlib inline

In [362]:
df_train = pd.read_csv("./content/train_dataset_train.csv")
df_train['split'] = 'train'
df_train_len = len(df_train)

In [363]:
df_test = pd.read_csv("./content/test_dataset_test.csv")
df_test['split'] = 'test'

In [364]:
df_with_split = pd.concat([df_train, df_test], axis=0)
#df_with_split = df_with_split.set_index('RecordNo')

## Рассмотрим датасет по ближе

In [379]:
df_with_split.head(3)

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,split,cultura_review_words_count,review_total_words_count
RecordNo,,,,,,,,,,,,,,,,,,,
6145,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0.0,0.0,0.0,0.0,0.0,train,0,15
7006,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0.0,0.0,0.0,0.0,1.0,train,2,28
1124,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0.0,1.0,0.0,0.0,1.0,train,4,31


In [198]:
df_with_split.shape

(6914, 17)

In [10]:
df_with_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4839 entries, 0 to 4838
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   RecordNo                              4839 non-null   int64  
 1   Название книги                        4839 non-null   object 
 2   Автор                                 4839 non-null   object 
 3   Ссылка на литрес                      4839 non-null   object 
 4   Рейтинг                               4839 non-null   float64
 5   Количество оценок                     4839 non-null   int64  
 6   Количество отзывов                    4839 non-null   int64  
 7   Имя читателя                          4839 non-null   object 
 8   Оценка книги читателем (из 5 баллов)  4070 non-null   float64
 9   Отзыв                                 4839 non-null   object 
 10  Лайки на отзыв                        4839 non-null   int64  
 11  Дислайки на отзыв

## Добавим новые признаки

In [396]:
df_with_split["len_reviews"] = df_with_split["Отзыв"].map(len)

In [201]:
df_with_split["len_reviews"].describe()

count    6914.000000
mean      337.263379
std       364.006785
min         1.000000
25%       170.000000
50%       235.000000
75%       377.000000
max      8402.000000
Name: len_reviews, dtype: float64

In [397]:
df_with_split["Автор"] = pd.Categorical(df_with_split["Автор"])
df_with_split["Автор"].astype('category').cat.codes
df_with_split["Автор"] = df_with_split["Автор"].cat.codes
df_with_split["Автор"].astype('int64')

RecordNo
6145    23
7006    23
1124    23
4716    27
2946    27
        ..
4661    23
1547    23
6071    23
3805    45
471     48
Name: Автор, Length: 6914, dtype: int64

In [381]:
df_with_split

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,split,cultura_review_words_count,review_total_words_count
RecordNo,,,,,,,,,,,,,,,,,,,
6145,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0.0,0.0,0.0,0.0,0.0,train,0,15
7006,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0.0,0.0,0.0,0.0,1.0,train,2,28
1124,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0.0,1.0,0.0,0.0,1.0,train,4,31
4716,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0.0,1.0,0.0,0.0,0.0,train,0,39
2946,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0.0,0.0,0.0,0.0,0.0,train,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4661,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Екатерина Юзбашьян,5.0,"Очень понравилась!!!! Написана грамотным, крас...",0,1,NaN,NaN,NaN,NaN,NaN,test,1,26
1547,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,239833928,4.0,Зулейха понравилась больше. Книга Дети мои каж...,0,0,NaN,NaN,NaN,NaN,NaN,test,0,26
6071,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,I be,5.0,"Прочла на одном дыхании, не смотря на раздираю...",0,1,NaN,NaN,NaN,NaN,NaN,test,4,37


In [203]:
df_with_split = df_with_split.fillna(0)

In [286]:
# Подготавливаем список стоп-слов
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
words_stop = stopwords.words('russian')

# добавляем те слова и знаки которые мы так же считаем не нужными
st = ['.',',','!','?','-', '/', '(', ')']
words_stop +=st
words_stop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

## Обрабатываем колонку 'Название книги' на предмет там нахождения слов, имеющих отношение к ценности

In [287]:
#!pip install razdel

In [288]:
# Разделяем на слова
from razdel import tokenize

from tqdm import tqdm

def pd_tokenize(col_name): 
    
    text_strings = []

    for index, row in tqdm(df_with_split.iterrows()): 
        text_list = list(tokenize(row[col_name]))
        for i_tokeniz in text_list:
            if i_tokeniz.text not in words_stop:
                text_strings.append((index, i_tokeniz.text))

    # для удобства преобразуем в датафрейм
    df_tokenize = pd.DataFrame(text_strings,columns=['RecordNo', 'tokenize'])   
    #df_tokenize['RecordNo', 'tokenize'] = text_strings
    return df_tokenize

### Подготавливаем синонимические ряды

In [290]:
'''
традиционным
российским духовно-нравственным ценностям относятся «жизнь,
достоинство, права и свободы человека, патриотизм, гражданственность,
служение Отечеству и ответственность за его судьбу, высокие нравственные
идеалы, крепкая семья, созидательный труд, приоритет духовного над
материальным, гуманизм, милосердие, справедливость, коллективизм,
взаимопомощь и взаимоуважение, историческая память и преемственность
поколений, единство народов России»
'''

#https://synonymonline.ru/
    
cultura_words0 = [
    "традиционный", "российский", "духовный", "нравственный", "ценность",
    "жизнь", "достоинство", "право", "свобода", "человек", "патриотизм", "гражданственность",
    "служение", "отечество", "ответственность", "судьба", "высокий", "нравственный", "идеал", "крепкий", "семья", 
    "созидательный", "труд", "приоритет", "духовный", "материальный", "гуманизм", "милосердие", "справедливость", 
    "коллективизм", "взаимопомощь", "взаимоуважение", "исторический", "память", "преемственность", "поколение", 
    "единство", "народ", "россия",
]

cultura_word1 = [ # ценность
    "важность", "достоинство", "значение", "цена", "стоимость", "вес", "преимущество", "смысл", "сила", "авторитетность",
    "драгоценность", "неоценимость", "многоценность", "сверхценность", "самоценность", "традиция", "значимость", 
    "клад", "нужность", "неоцененность", "сокровище", "перл", "многознаменательность", "аксиология"
]

cultura_word2 = [ # традиционный
    "обычный", "сакраментальный", "установленный", "установившийся", "устоявшийся", "укоренившийся"
]

cultura_word4 = [ # нравственный
    "высоконравственный", "честный", "целомудренный", "добродетельный", "духовный", "душевный", "моральный", 
    "этический", "этичный", "поучительный", "добрый"
]    

cultura_word5 = [ # достоинство
    "преимущество", "величие", "величавость", "важность", "гордость"
]

cultura_word6 = [ # право
    "прерогатива", "преимущество", "привилегия", "льгота" 
]

cultura_word7 = [ # свобода
    "воля", "независимость", "беспрепятственность"
]    

cultura_word8 = [ # человек
    "личность", "особа", "персона", "индивид", "индивидуум", 
]    

cultura_word9 = [ # Отечество
    "отчизна", "родина", "страна" 
]    

cultura_word10 = [ # ответственность
    "ручательство", "поручительство", "порука", "обязательство", "надежность", "обязательность" 
]    

cultura_word11 = [ # судьба
    "предопределение", "провидение", "будущность", "грядущее", "удел", "жребий", "доля", "участь", "предназначение",
]    

cultura_word12 = [ # идеал
    "образец", "пример", "цель", "эталон", "совершенство", "абсолют"
]

cultura_word13 = [ # семья
    "родные", "род", "династия"
]    

cultura_word14 = [ # труд
    "работа", "дело", "деятельность", "усилие", "труженичество", "занятие", "старание", "напряжение", "создание", "творение",
]    

cultura_word15 = [ # приоритет
    "преимущество", "первенство"
]

cultura_word3 = [ # гуманизм
    "мировоззрение", "гуманность", "человеколюбие", "человечность"
]    

cultura_word16 = [ # милосердие
    "добросердечие", "доброта", "сочувствие",
    "благожелательность", "благость", "благоволение", "доброжелательность",
    "благосклонность", "любезность", "великодушие", "жалость", "сострадание", "ласка",
    "любовь", "снисхождение", "добро", "сердоболие", "прощение"
]    

cultura_word17 = [ # cправедливость
    "праведность", "честность", "беспристрастность",
    "беспристрастие", "заслуженность", "нелицеприятность", "непредубежденность",
    "непредвзятость", "объективность", "правда", "правильность", "верность", "законность",
    "достоверность", "истинность", "правосудие"
]    

cultura_word18 = [ # коллективизм
    "содружество"
]    

cultura_word19 = [ # взаимопомощь
    "помощь", "взаимовыручка"
]    

cultura_word20 = [ # взаимоуважение
    "уважение"
]    

cultura_word21 = [ # народ
    "люди", "население", "нация", "народонаселение"
]

cultura_word22 = [ # единство
    "целостность", "цельность", "монолитность",
    "целость", "общность", "соборность", "идентичность", "слитность", "тождественность",
    "сплоченность", "солидарность", "единомыслие", "единогласие", "спаянность",
    "гармония", "гармоничность", "мир", "равенство", "тождество", "согласие", "сходство"
]    

cultura_words = set(cultura_words0) |\
                set(cultura_word1) | set(cultura_word2) | set(cultura_word3) | set(cultura_word4) | set(cultura_word5) |\
                set(cultura_word6) | set(cultura_word7) | set(cultura_word8) | set(cultura_word9) | set(cultura_word10) |\
                set(cultura_word11) | set(cultura_word12) | set(cultura_word13) | set(cultura_word14) | set(cultura_word15) |\
                set(cultura_word16) | set(cultura_word17) | set(cultura_word18) | set(cultura_word19) | set(cultura_word20) |\
                set(cultura_word21) | set(cultura_word22)
                                                                                                      

### Проводим лемматизацию с помощью natasha

In [291]:
#!pip install natasha

## Обрабатываем колонку 'Отзыв' на предмет там нахождения слов, имеющих отношение к ценности

In [316]:
df_tokenize = pd_tokenize('Отзыв')

6914it [00:04, 1518.99it/s]


In [317]:
df_tokenize

,RecordNo,tokenize
0,6145,Рекомендую
1,6145,книгу
2,6145,прочтению
3,6145,прослушиванию
4,6145,Есть
...,...,...
255900,471,откуда
255901,471,взяты
255902,471,образы
255903,471,известных


In [305]:
df_tokenize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255905 entries, 0 to 255904
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   RecordNo  255905 non-null  int64 
 1   tokenize  255905 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.9+ MB


In [338]:
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
   
    PER,
    Doc
)

def get_lemmization(df_tokenize):
    
    segmenter = Segmenter()
    morph_vocab = MorphVocab()

    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)

    cultura_result =[]
    
    # Токенизируем и разбиваем на леммы
    for index, row in tqdm(df_tokenize.iterrows()): 
        text = row['tokenize']
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
    
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
        lemms = []
        for doc_token  in doc.tokens:
            if doc_token.lemma in cultura_words:
                lemms.append(doc_token.lemma)

        if len(lemms) > 0:
            cultura_result.append((index, row['RecordNo'], lemms))
    return cultura_result

In [339]:
cultura_result = get_lemmization(df_tokenize)

255905it [16:23, 260.08it/s]


In [340]:
cultura_result

[(26, 7006, ['жизнь']),
 (30, 7006, ['мир']),
 (55, 1124, ['смысл']),
 (59, 1124, ['семья']),
 (64, 1124, ['сила']),
 (71, 1124, ['мир']),
 (128, 2946, ['высокий']),
 (248, 7635, ['жизнь']),
 (251, 7635, ['человек']),
 (275, 7226, ['жизнь']),
 (316, 7226, ['жизнь']),
 (317, 7226, ['россия']),
 (400, 3276, ['правда']),
 (466, 5321, ['напряжение']),
 (523, 3383, ['мир']),
 (541, 3383, ['ценность']),
 (546, 3383, ['род']),
 (646, 80, ['память']),
 (649, 80, ['традиция']),
 (652, 80, ['родные']),
 (661, 80, ['человек']),
 (693, 80, ['человек']),
 (695, 80, ['родные']),
 (705, 80, ['моральный']),
 (709, 80, ['человек']),
 (713, 80, ['жизнь']),
 (717, 80, ['усилие']),
 (744, 80, ['дело']),
 (755, 80, ['жизнь']),
 (756, 80, ['человек']),
 (758, 80, ['родина']),
 (761, 80, ['уважение']),
 (762, 80, ['ценность']),
 (770, 4990, ['жизнь']),
 (805, 4629, ['жизнь']),
 (807, 4629, ['сила']),
 (808, 4629, ['любовь']),
 (848, 6554, ['человек']),
 (871, 4843, ['судьба']),
 (872, 4843, ['страна']),
 (87

In [341]:
df_cultura = pd.DataFrame(cultura_result,columns=['index', 'tokenize_id',  'cultura_review_words'])   

In [342]:
len(df_cultura)

10852

In [343]:
df_cultura

,index,tokenize_id,cultura_review_words
0,26,7006,[жизнь]
1,30,7006,[мир]
2,55,1124,[смысл]
3,59,1124,[семья]
4,64,1124,[сила]
...,...,...,...
10847,255853,6071,[любовь]
10848,255871,3805,[любовь]
10849,255888,471,[семья]
10850,255891,471,[жизнь]


In [344]:
df_tokenize['index'] = df_tokenize.index

In [345]:
df_tokenize

,RecordNo,tokenize,index
0,6145,Рекомендую,0
1,6145,книгу,1
2,6145,прочтению,2
3,6145,прослушиванию,3
4,6145,Есть,4
...,...,...,...
255900,471,откуда,255900
255901,471,взяты,255901
255902,471,образы,255902
255903,471,известных,255903


In [346]:
df_cultura_result = df_tokenize.merge(df_cultura, left_on='index', right_on='index', how='left' )

In [348]:
df_cultura_result['review_total_words'] = 1

In [349]:
df_cultura_result

,RecordNo,tokenize,index,tokenize_id,cultura_review_words,review_total_words
0,6145,Рекомендую,0,NaN,NaN,1
1,6145,книгу,1,NaN,NaN,1
2,6145,прочтению,2,NaN,NaN,1
3,6145,прослушиванию,3,NaN,NaN,1
4,6145,Есть,4,NaN,NaN,1
...,...,...,...,...,...,...
255900,471,откуда,255900,NaN,NaN,1
255901,471,взяты,255901,NaN,NaN,1
255902,471,образы,255902,NaN,NaN,1
255903,471,известных,255903,NaN,NaN,1


In [350]:
df_cultura_group = df_cultura_result.groupby('RecordNo')['cultura_review_words','review_total_words'].agg('count').add_suffix("_count").reset_index()

C:\Users\Сергей\AppData\Local\Temp\ipykernel_15652\2604123461.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cultura_group = df_cultura_result.groupby('RecordNo')['cultura_review_words','review_total_words'].agg('count').add_suffix("_count").reset_index()


In [365]:
df_cultura_group

,RecordNo,cultura_review_words_count,review_total_words_count
0,2,2,69
1,3,0,8
2,6,4,119
3,7,2,30
4,8,1,47
...,...,...,...
6909,7954,2,20
6910,7955,2,21
6911,7957,1,21
6912,7958,2,48


In [366]:
df_with_split = df_with_split.merge(df_cultura_group, left_on='RecordNo', right_on='RecordNo', how='left' )

In [369]:
df_with_split = df_with_split.set_index('RecordNo')

In [384]:
df_with_split

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,split,cultura_review_words_count,review_total_words_count
RecordNo,,,,,,,,,,,,,,,,,,,
6145,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0.0,0.0,0.0,0.0,0.0,train,0,15
7006,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0.0,0.0,0.0,0.0,1.0,train,2,28
1124,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0.0,1.0,0.0,0.0,1.0,train,4,31
4716,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0.0,1.0,0.0,0.0,0.0,train,0,39
2946,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0.0,0.0,0.0,0.0,0.0,train,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4661,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Екатерина Юзбашьян,5.0,"Очень понравилась!!!! Написана грамотным, крас...",0,1,NaN,NaN,NaN,NaN,NaN,test,1,26
1547,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,239833928,4.0,Зулейха понравилась больше. Книга Дети мои каж...,0,0,NaN,NaN,NaN,NaN,NaN,test,0,26
6071,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,I be,5.0,"Прочла на одном дыхании, не смотря на раздираю...",0,1,NaN,NaN,NaN,NaN,NaN,test,4,37


## Выделим выборки

In [425]:
mass_label = ['Релевантность',
       'Таксономия релевантные', 'Таксономия не релевантные',
       'Длина отзыва', 'Ценности']

In [426]:
object_label = df_with_split.dtypes[df_with_split.dtypes == "object"].index.values

In [427]:
object_label = ['Название книги', 'Ссылка на литрес', 'Имя читателя', 'Отзыв', 'split']

In [428]:
df_train = df_with_split.loc[df_with_split['split'] == 'train']
df_train.head()

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,split,cultura_review_words_count,review_total_words_count,len_reviews
RecordNo,,,,,,,,,,,,,,,,,,,,
6145,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0.0,0.0,0.0,0.0,0.0,train,0,15,157
7006,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0.0,0.0,0.0,0.0,1.0,train,2,28,283
1124,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0.0,1.0,0.0,0.0,1.0,train,4,31,278
4716,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0.0,1.0,0.0,0.0,0.0,train,0,39,342
2946,ПОСТ,27,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0.0,0.0,0.0,0.0,0.0,train,1,20,177


In [446]:
df_train = df_train.fillna(0)

In [447]:
X = df_train.drop(mass_label, axis = 1)
X = X.drop(object_label, axis = 1)

y = df_train[mass_label]

In [448]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4839 entries, 6145 to 6849
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Автор                                 4839 non-null   int8   
 1   Рейтинг                               4839 non-null   float64
 2   Количество оценок                     4839 non-null   int64  
 3   Количество отзывов                    4839 non-null   int64  
 4   Оценка книги читателем (из 5 баллов)  4839 non-null   float64
 5   Лайки на отзыв                        4839 non-null   int64  
 6   Дислайки на отзыв                     4839 non-null   int64  
 7   cultura_review_words_count            4839 non-null   int64  
 8   review_total_words_count              4839 non-null   int64  
 9   len_reviews                           4839 non-null   int64  
dtypes: float64(2), int64(7), int8(1)
memory usage: 382.8 KB


In [449]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели

In [450]:
clf = RandomForestClassifier(random_state=0)

In [451]:
clf.fit(X, y)

RandomForestClassifier(random_state=0)

## Оценка точности

In [435]:
#pred = clf.predict(X_test)

In [436]:
#y_test.head(3)

In [437]:
#from sklearn.metrics import recall_score

#print("Recall score:", recall_score(y_test, pred, average='macro'))

## Выводим результат

In [452]:
df_test = df_with_split.loc[df_with_split['split'] == 'test']
df_test.head()

,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности,split,cultura_review_words_count,review_total_words_count,len_reviews
RecordNo,,,,,,,,,,,,,,,,,,,,
3366,Дети мои,23,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Марина Ефимкина,5.0,"Настоящая глубокая книга, коих сейчас очень не...",2,0,NaN,NaN,NaN,NaN,NaN,test,2,20,181
3952,Текст,27,https://www.litres.ru/dmitriy-gluhovskiy/tekst...,4.5,1923,246,alexvarp,5.0,Одна из лучших книг прочитанных за последние г...,1,0,NaN,NaN,NaN,NaN,NaN,test,0,22,219
6852,Текст,27,https://www.litres.ru/dmitriy-gluhovskiy/tekst/,4.4,7276,622,fb_154207611938008,5.0,"Тот случай, когда невозможно пройти мимо, не о...",0,0,NaN,NaN,NaN,NaN,NaN,test,1,45,390
4586,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Мария Курочкина,NaN,"Очень не глубокая книга. Дочитала до конца, в ...",4,0,NaN,NaN,NaN,NaN,NaN,test,1,34,280
4677,Зулейха открывает глаза,23,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,sahnovavera,NaN,"Очень понравилась. Читается легко, о непростых...",0,1,NaN,NaN,NaN,NaN,NaN,test,2,16,136


In [453]:
X_test = df_test.drop(mass_label, axis = 1)
X_test = X_test.drop(object_label, axis = 1)
X_test

,Автор,Рейтинг,Количество оценок,Количество отзывов,Оценка книги читателем (из 5 баллов),Лайки на отзыв,Дислайки на отзыв,cultura_review_words_count,review_total_words_count,len_reviews
RecordNo,,,,,,,,,,
3366,23,4.4,8032,702,5.0,2,0,2,20,181
3952,27,4.5,1923,246,5.0,1,0,0,22,219
6852,27,4.4,7276,622,5.0,0,0,1,45,390
4586,23,4.6,24719,2103,NaN,4,0,1,34,280
4677,23,4.6,24719,2103,NaN,0,1,2,16,136
...,...,...,...,...,...,...,...,...,...,...
4661,23,4.6,24719,2103,5.0,0,1,1,26,243
1547,23,4.4,8032,702,4.0,0,0,0,26,186
6071,23,4.6,24719,2103,5.0,0,1,4,37,324


In [454]:
X_test = X_test.fillna(0)

In [455]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 3366 to 471
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Автор                                 2075 non-null   int8   
 1   Рейтинг                               2075 non-null   float64
 2   Количество оценок                     2075 non-null   int64  
 3   Количество отзывов                    2075 non-null   int64  
 4   Оценка книги читателем (из 5 баллов)  2075 non-null   float64
 5   Лайки на отзыв                        2075 non-null   int64  
 6   Дислайки на отзыв                     2075 non-null   int64  
 7   cultura_review_words_count            2075 non-null   int64  
 8   review_total_words_count              2075 non-null   int64  
 9   len_reviews                           2075 non-null   int64  
dtypes: float64(2), int64(7), int8(1)
memory usage: 164.1 KB


In [456]:
y = df_test[mass_label]

In [457]:
y

,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
RecordNo,,,,,
3366,NaN,NaN,NaN,NaN,NaN
3952,NaN,NaN,NaN,NaN,NaN
6852,NaN,NaN,NaN,NaN,NaN
4586,NaN,NaN,NaN,NaN,NaN
4677,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
4661,NaN,NaN,NaN,NaN,NaN
1547,NaN,NaN,NaN,NaN,NaN
6071,NaN,NaN,NaN,NaN,NaN


In [458]:
pred = clf.predict(X_test)

In [459]:
pred

array([[0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1.],
       ...,
       [1., 1., 0., 0., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [460]:
y[mass_label] = pred

C:\Users\Сергей\AppData\Local\Temp\ipykernel_15652\124737008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[mass_label] = pred


In [461]:
y

,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
RecordNo,,,,,
3366,0.0,1.0,0.0,0.0,1.0
3952,0.0,1.0,0.0,0.0,0.0
6852,0.0,1.0,0.0,0.0,1.0
4586,0.0,1.0,0.0,0.0,1.0
4677,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
4661,0.0,1.0,0.0,0.0,1.0
1547,1.0,1.0,0.0,0.0,0.0
6071,1.0,1.0,0.0,0.0,1.0


In [462]:
df_submission = y.reset_index()
df_submission

,RecordNo,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
0,3366,0.0,1.0,0.0,0.0,1.0
1,3952,0.0,1.0,0.0,0.0,0.0
2,6852,0.0,1.0,0.0,0.0,1.0
3,4586,0.0,1.0,0.0,0.0,1.0
4,4677,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...
2070,4661,0.0,1.0,0.0,0.0,1.0
2071,1547,1.0,1.0,0.0,0.0,0.0
2072,6071,1.0,1.0,0.0,0.0,1.0
2073,3805,0.0,0.0,0.0,1.0,1.0


In [463]:
df_submission.to_csv("./content/RSF_submission1.csv", index=False)